# 📍 Geolocalizador de Negocios por Código Postal
Este notebook permite consultar negocios por código postal usando la API de Google Places y visualizar los resultados en un mapa interactivo con Folium.

## 🔑 Configurar clave de API

In [12]:
import os
from dotenv import load_dotenv

# Carga clave desde .env o asignación directa temporal
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

## ⚙️ Funciones auxiliares

In [13]:
import requests
import pandas as pd
import time

def get_coordinates(postal_code):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': f'{postal_code}, Madrid, Spain', 'key': API_KEY}
    response = requests.get(url, params=params).json()
    
    if response['status'] != 'OK':
        raise ValueError(f"Error al obtener coordenadas para {postal_code}: {response['status']}")
    
    result = response['results'][0]
    
    # Validar que el país devuelto sea España mediante short_name = 'ES'
    components = result['address_components']
    country_codes = [c['short_name'] for c in components if 'country' in c['types']]
    if 'ES' not in country_codes:
        raise ValueError(f"Código postal {postal_code} no está en España (devuelto: {country_codes})")
    
    location = result['geometry']['location']
    return location['lat'], location['lng']

def get_places(lat, lng, radius=2000, business_type=None):
    places = []
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': radius,
        'key': API_KEY,
        'type': business_type
    }

    while True:
        res = requests.get(url, params=params).json()
        places.extend(res.get('results', []))
        if 'next_page_token' in res:
            time.sleep(2)
            params['pagetoken'] = res['next_page_token']
        else:
            break

    return places

def clean_data(places, postal_code):
    data = []
    for p in places:
        location = p.get('geometry', {}).get('location', {})
        data.append({
            'nombre': p.get('name'),
            'direccion': p.get('vicinity'),
            'codigo_postal': postal_code,
            'puntuacion_media': p.get('rating'),
            'numero_reviews': p.get('user_ratings_total'),
            'tipo_negocio': ', '.join(p.get('types', [])),
            'latitud': location.get('lat'),
            'longitud': location.get('lng')
        })
    return pd.DataFrame(data)

## 📦 Consultar locales y exportar CSV

In [ ]:

import time
import pandas as pd

codigos_postales = ['28001', '28002', '28003', '28004', '28005', '28006', '28007', '28008', '28009',
                    '28010', '28011', '28012', '28013', '28014', '28015', '28016', '28017', '28018', '28019',
                    '28020', '28021', '28022', '28023', '28024', '28025', '28026', '28027', '28028', '28029',
                    '28030', '28031', '28032', '28033', '28034', '28035', '28036', '28037', '28038', '28039',
                    '28040', '28041', '28042', '28043', '28044', '28045', '28046', '28047', '28048', '28049',
                    '28050', '28051', '28052', '28053', '28054', '28055']
tipo_negocio = None  # Puedes especificar un tipo como 'restaurant', 'store', etc.

todos_los_locales = pd.DataFrame()

for cp in codigos_postales:
    try:
        lat, lng = get_coordinates(cp)
        lugares = get_places(lat, lng, business_type=tipo_negocio)
        df = clean_data(lugares, cp)
        todos_los_locales = pd.concat([todos_los_locales, df], ignore_index=True)
        print(f"✅ {len(df)} locales encontrados en {cp}")
        time.sleep(1)
    except ValueError as e:
        print(f"⚠️ Error de validación con código postal {cp}: {e}")
    except Exception as e:
        print(f"⚠️ Error inesperado con código postal {cp}: {e}")

# Guardar el archivo con el nombre esperado
nombre_tipo = tipo_negocio if tipo_negocio else "todos"
output_file = f"locales_{nombre_tipo}.csv"
todos_los_locales.to_csv(output_file, index=False)
print(f"\n✅ Archivo '{output_file}' generado exitosamente.")


✅ 60 locales encontrados en 28001


## 🗺️ Visualizar resultados en mapa interactivo

In [6]:
import folium.plugins as plugins
import folium

df = pd.read_csv("data/locales_todos.csv")
df = df.dropna(subset=['latitud', 'longitud'])

mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'  # Estilos --> https://leaflet-extras.github.io/leaflet-providers/preview/
    )

# Añadir MiniMap
plugins.MiniMap().add_to(mapa)

for _, row in df.iterrows():
    popup_text = f"""
    <b>{row['nombre']}</b><br>
    Dirección: {row['direccion']}<br>
    Rating: {row['puntuacion_media']} ({row['numero_reviews']} reseñas)<br>
    Tipo: {row['tipo_negocio']}
    """
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        icon=plugins.BeautifyIcon(
                     icon="home",
                     icon_shape="circle",
                     border_color='green',
                     text_color="green",
                     background_color='white'
                 ),
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['nombre']
    ).add_to(mapa)

mapa.save("outputs/mapa_locales.html")
mapa


FileNotFoundError: [Errno 2] No such file or directory: 'data/locales_todos.csv'

### Categorización de tipos de negocio
En esta sección se analiza la columna `tipo_negocio` del archivo `locales_todos.csv` para agrupar los locales en 6 categorías generales:
- **Restauración**
- **Alojamiento**
- **Religión / Culto**
- **Administración / Localidad**
- **Comercio / Servicios**
- **Otros / No categorizado**
Esto se realiza mediante una función que evalúa palabras clave en cada tipo de negocio y asigna una categoría correspondiente.

### Categorización mejorada de tipos de negocio
Se han identificado subtipos frecuentes dentro de la categoría 'Otros / No categorizado' y se han creado nuevas agrupaciones:
- **Educación**
- **Salud**
- **Finanzas y Seguros**
- **Construcción / Reformas**
- **Naturaleza / Recreación**
- **Turismo / Atracción**
Esto reduce significativamente el número de registros sin categorizar y mejora la calidad del dataset para futuros modelos de Machine Learning.

In [ ]:
import pandas as pd

df = pd.read_csv('data/locales_todos.csv')

def categorizar_negocio(tipo_str):
    tipo_str = str(tipo_str).lower()
    if any(t in tipo_str for t in ['restaurant', 'food', 'bar', 'cafe', 'bakery', 'meal_takeaway']):
        return 'Restauración'
    elif any(t in tipo_str for t in ['lodging', 'hotel', 'hostel', 'guest_house']):
        return 'Alojamiento'
    elif any(t in tipo_str for t in ['church', 'place_of_worship', 'mosque', 'synagogue', 'hindu_temple']):
        return 'Religión / Culto'
    elif any(t in tipo_str for t in ['locality', 'political', 'city_hall', 'post_office', 'courthouse', 'embassy', 'local_government_office']):
        return 'Administración / Localidad'
    elif any(t in tipo_str for t in ['store', 'shopping_mall', 'hair_care', 'bank', 'car_repair', 'laundry', 'gym', 'pharmacy', 'doctor', 'real_estate_agency']):
        return 'Comercio / Servicios'
    elif any(t in tipo_str for t in ['school', 'university', 'secondary_school']):
        return 'Educación'
    elif any(t in tipo_str for t in ['health']):
        return 'Salud'
    elif any(t in tipo_str for t in ['finance', 'insurance_agency']):
        return 'Finanzas y Seguros'
    elif any(t in tipo_str for t in ['general_contractor']):
        return 'Construcción / Reformas'
    elif any(t in tipo_str for t in ['park']):
        return 'Naturaleza / Recreación'
    elif any(t in tipo_str for t in ['tourist_attraction']):
        return 'Turismo / Atracción'
    else:
        return 'Otros / No categorizado'

df['categoria_negocio'] = df['tipo_negocio'].apply(categorizar_negocio)
df.to_csv('data/MAPS_locales_categorizados.csv', index=False)

### 🔍 1.1 - Añadir detección de negocio por nombre

Ahora que tienes una categorización robusta por tipo_negocio, puedes mejorar la clasificación usando el campo nombre para los que aún están en “Otros”. Por ejemplo:

In [ ]:
def clasificar_por_nombre(nombre, tipo_actual):
    nombre = str(nombre).lower()
    if tipo_actual == 'Otros / No categorizado':
        if 'bar' in nombre: return 'Restauración'
        if 'peluquería' in nombre or 'hair' in nombre: return 'Comercio / Servicios'
        if 'taller' in nombre or 'auto' in nombre: return 'Comercio / Servicios'
        # ... y así puedes mejorar hasta un 20% más de casos
    return tipo_actual

df['categoria_negocio'] = df.apply(lambda row: clasificar_por_nombre(row['nombre'], row['categoria_negocio']), axis=1)

### 🎯 1.2 - Filtrar outliers

Elimina negocios con:
	•	puntuacion_media nula o 0
	•	numero_reviews muy bajo (ej. < 3)
Esto limpia el ruido antes de entrenar modelos o hacer visualizaciones.

In [ ]:
df = df[df['puntuacion_media'].notna() & (df['numero_reviews'] >= 3)]

### 📦 1.3 - Guardar checkpoint de trabajo

Para no repetir procesos:

In [ ]:
df.to_csv('data/MAPS_locales_procesado.csv', index=False)

### Paso 2: Crear métrica de éxito del negocio
Se calcula una puntuación ponderada que penaliza los negocios con pocas reseñas usando la fórmula sugerida por el profesor.

In [ ]:
import numpy as np

df['valoracion'] = df.apply(
    lambda row: row['puntuacion_media'] * (1 - np.exp(-row['numero_reviews'] / 10)), axis=1
)

### Paso 3: Normalización de valoraciones por tipo de negocio
La métrica de éxito se normaliza dentro de cada categoría de negocio para que los valores estén entre 0 y 1.

In [ ]:

from sklearn.preprocessing import MinMaxScaler

df['valoracion_norm'] = df.groupby('categoria_negocio')['valoracion'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)


### Paso 4: Visualización geoespacial del éxito de los negocios
Se crea un mapa de calor utilizando la puntuación normalizada para observar zonas con mayor concentración de negocios exitosos.

In [ ]:
import folium
from folium import plugins
import branca.colormap as cm

colormap = cm.LinearColormap(['red', 'orange', 'green'], vmin=1, vmax=5)
colormap.caption = 'Valoración de los negocios'

mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'  # Estilos --> https://leaflet-extras.github.io/leaflet-providers/preview/
    )


# Añadir MiniMap
plugins.MiniMap().add_to(mapa)

# Añadir marcadores como nube de puntos
for _, row in df.iterrows():
    color = colormap(row['valoracion'])
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"{row['nombre']} ({row['valoracion']:.1f})"
    ).add_to(mapa)

colormap.add_to(mapa)

mapa

In [ ]:
import folium
from folium.plugins import HeatMap, MiniMap
import branca.colormap as cm

# Crear el mapa base
mapa_densidad = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# Añadir el MiniMap
MiniMap().add_to(mapa_densidad)

# Preparar datos para el mapa de calor (sin valoración)
heat_data = [[row['latitud'], row['longitud']] for index, row in df.iterrows()]

# Añadir el mapa de calor
HeatMap(
    heat_data,
    radius=15,
    blur=12,
    min_opacity=0.3,
    max_zoom=1
).add_to(mapa_densidad)

# Crear una leyenda simple de densidad
colormap = cm.LinearColormap(
    colors=['lightblue', 'orange', 'red'],
    index=[0, 0.5, 1],
    vmin=0,
    vmax=1
)
colormap.caption = 'Densidad de negocios (baja a alta)'
colormap.add_to(mapa_densidad)

# Mostrar el mapa
mapa_densidad

### Paso 5: Clustering geoespacial con DBSCAN
Se aplica DBSCAN para identificar agrupaciones de zonas con negocios exitosos y etiquetar los puntos con un número de clúster.

In [ ]:

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

coords = df[['latitud', 'longitud']].values
coords_scaled = StandardScaler().fit_transform(coords)
db = DBSCAN(eps=0.3, min_samples=10).fit(coords_scaled)
df['cluster'] = db.labels_


### 🌍 PASO 6: INCORPORAR VARIABLES EXTERNAS


In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MiniMap
import branca.colormap as cm
from shapely.geometry import Point

# 1. Cargar el shapefile de barrios de Madrid
gdf_barrios = gpd.read_file('/data/Barrios/BARRIOS.shp')

# 2. Convertir tu DataFrame df en GeoDataFrame
gdf_negocios = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud), crs='EPSG:4326')

# 3. Unión espacial para asignar barrio a cada negocio
gdf_completo = gpd.sjoin(gdf_negocios, gdf_barrios, how='left', predicate='within')

# 4. Contar negocios por barrio (usando la columna 'NOMBRE')
conteo = gdf_completo.groupby('NOMBRE').size().reset_index(name='num_negocios')
gdf_barrios = gdf_barrios.merge(conteo, on='NOMBRE', how='left')
gdf_barrios['num_negocios'] = gdf_barrios['num_negocios'].fillna(0)

# 5. Normalizar la densidad
gdf_barrios['densidad_norm'] = (
    gdf_barrios['num_negocios'] - gdf_barrios['num_negocios'].min()
) / (
    gdf_barrios['num_negocios'].max() - gdf_barrios['num_negocios'].min()
)

# 6. Crear el mapa
mapa_barrios = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=12,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# 7. Añadir MiniMap
MiniMap().add_to(mapa_barrios)

# 8. Leyenda de colores
colormap = cm.LinearColormap(['lightblue', 'orange', 'red'], vmin=0, vmax=1)
colormap.caption = 'Densidad normalizada de negocios por barrio'
colormap.add_to(mapa_barrios)

# 9. Dibujar polígonos de barrios
folium.GeoJson(
    gdf_barrios,
    name="Barrios por densidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['densidad_norm']),
        'color': 'black',
        'weight': 0.7,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMBRE", "num_negocios"],
        aliases=["Barrio", "Nº negocios"],
        localize=True
    )
).add_to(mapa_barrios)

# 10. Mostrar mapa
mapa_barrios

ModuleNotFoundError: No module named 'geopandas'